In [1]:
from mintpy.utils import isce_utils
import os
from mintpy import multilook
from mintpy.utils import utils as ut
from mintpy.utils import ptime, readfile, writefile, isce_utils, utils as ut, plot as pp, attribute as attr
from datetime import datetime

# 初始SLC的路径
SLC_dir = './merged/SLC/'

# 获取所有日期
dates = []
for folder_name in os.listdir(SLC_dir):
    folder_path = os.path.join(SLC_dir, folder_name)
    if os.path.isdir(folder_path):
        # 检查文件夹是否为空
        if os.listdir(folder_path):
            try:
                date = folder_name.strip()[:8]  # 提取前8个字符作为日期（YYYYMMDD）
                dates.append(date)
            except IndexError:
                pass
dates = sorted(dates)
print(dates)

# 把日期字符串转为datetime对象方便计算时间差
dates_dt = [datetime.strptime(d, '%Y%m%d') for d in dates]

# 生成第一类对, 针对每一个日期，生成其后10个日期的对
all_pairs = []

# 生成第二类对,相隔9-15个月的对. 用一个月平均为30天来计算时间差.
for i in range(len(dates_dt)):
    for j in range(i + 1, len(dates_dt)):
        duration = (dates_dt[j] - dates_dt[i]).days
        if 270 < duration < 450:  #270 days is about 9 months, and 450 days is about 15 months
            pair = f"{dates[i]}_{dates[j]}"
            all_pairs.append(pair)

slc_1s = []  ## 干涉时间1，例如20210202
slc_2s = []  ## 干涉时间2，例如20210202
for pair in all_pairs:
    date1, date2 = pair.split('_')
    slc_1s.append(date1)
    slc_2s.append(date2)


ERROR 1: PROJ: proj_create_from_database: Open of /home/eedy/tools/mambaforge/envs/insar/share/proj failed


['20141019', '20141112', '20141206', '20150123', '20150216', '20150228', '20150324', '20150417', '20150604', '20150628', '20150815', '20150908', '20151002', '20151026', '20151119', '20151213', '20160106', '20160130', '20160223', '20160318', '20160411', '20160505', '20160529', '20160716', '20160809', '20160902', '20160926', '20161020', '20161113', '20161207', '20161231', '20170124', '20170205', '20170217', '20170301', '20170313', '20170325', '20170406', '20170418', '20170430', '20170512', '20170524', '20170605', '20170617', '20170629', '20170711', '20170723', '20170804', '20170816', '20170828', '20170909', '20170921', '20171003', '20171015', '20171027', '20171108', '20171120', '20171202', '20171214', '20171226', '20180107', '20180119', '20180131', '20180212', '20180224', '20180308', '20180320', '20180401', '20180413', '20180425', '20180507', '20180519', '20180531', '20180612', '20180624', '20180718', '20180730', '20180811', '20180823', '20180904', '20180916', '20180928', '20181010', '20

In [2]:
# 定义工作目录
work_dir = '/panda/eedy/jinsha/s1_a026'
azimuth_looks = 3
range_looks = 9
strength = 0.8

In [3]:
# 检查并创建 run_files_annual 文件夹
folder_path = 'run_files_annual'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 在 run_files_annual 文件夹中创建 run_13_generate_burst_igram 文件并写入内容
file_path = os.path.join(folder_path, 'run_13_generate_burst_igram')
with open(file_path, 'w') as file:
    for i in range(len(all_pairs)):  # 假设 num_pairs 是一些你定义的值
        line = f'SentinelWrapper.py -c {work_dir}/configs_annual/config_generate_igram_{all_pairs[i]}\n'
        file.write(line)

# 在 run_files_annual 文件夹中创建 run_14_merge_burst_igram 文件并写入内容
file_path = os.path.join(folder_path, 'run_14_merge_burst_igram')
with open(file_path, 'w') as file:
    for i in range(len(all_pairs)):  # 假设 num_pairs 是一些你定义的值
        line = f'SentinelWrapper.py -c {work_dir}/configs_annual/config_merge_igram_{all_pairs[i]}\n'
        file.write(line)

# 在 run_files_annual 文件夹中创建 run_15_merge_burst_igram 文件并写入内容
file_path = os.path.join(folder_path, 'run_15_filter_coherence')
with open(file_path, 'w') as file:
    for i in range(len(all_pairs)):  # 假设 num_pairs 是一些你定义的值
        line = f'SentinelWrapper.py -c {work_dir}/configs_annual/config_igram_filt_coh_{all_pairs[i]}\n'
        file.write(line)

# 在 run_files_annual 文件夹中创建 run_16_unwrap文件并写入内容
file_path = os.path.join(folder_path, 'run_16_unwrap')
with open(file_path, 'w') as file:
    for i in range(len(all_pairs)):  # 假设 num_pairs 是一些你定义的值
        line = f'SentinelWrapper.py -c {work_dir}/configs_annual/config_igram_unw_{all_pairs[i]}\n'
        file.write(line)

In [4]:
# 检查并创建 configs_annual 文件夹
folder_path = 'configs_annual'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# 在 configs_annual 文件夹中创建文件并写入run_13_内容
for i in range(len(all_pairs)):
    file_name = f'config_generate_igram_{all_pairs[i]}'
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'w') as file:
        file.write('[Common]\n')
        file.write('##########################\n')
        file.write('###################################\n')
        file.write('[Function-1]\n')
        file.write('generateIgram :\n')
        file.write(f'reference : {work_dir}/coreg_secondarys/{slc_1s[i]}\n')
        file.write(f'secondary : {work_dir}/coreg_secondarys/{slc_2s[i]}\n')
        file.write(f'interferogram : {work_dir}/interferograms/{all_pairs[i]}\n')
        file.write('flatten : False\n')
        file.write('interferogram_prefix : fine\n')
        file.write('overlap : False\n')
        file.write('###################################\n')

# 在 configs_annual 文件夹中创建文件并写入run_14_内容
for i in range(len(all_pairs)):
    file_name = f'config_merge_igram_{all_pairs[i]}'
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'w') as file:
        file.write('[Common]\n')
        file.write('##########################\n')
        file.write('###################################\n')
        file.write('[Function-1]\n')
        file.write('mergeBursts :\n')
        file.write(f'stack : {work_dir}/stack\n')
        file.write(f'inp_reference : {work_dir}/interferograms/{all_pairs[i]}\n')
        file.write(f'dirname : {work_dir}/interferograms/{all_pairs[i]}\n')
        file.write('name_pattern : fine*int\n')
        file.write(f'outfile : {work_dir}/merged/interferograms/{all_pairs[i]}/fine.int\n')
        file.write('method : top\n')
        file.write('aligned : True\n')
        file.write('valid_only : True\n')
        file.write('use_virtual_files : True\n')
        file.write('multilook : True\n')
        file.write(f'range_looks : {range_looks}\n')
        file.write(f'azimuth_looks : {azimuth_looks}\n')

# 在 configs_annual 文件夹中创建文件并写入run_15_内容
for i in range(len(all_pairs)):
    file_name = f'config_igram_filt_coh_{all_pairs[i]}'
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'w') as file:
        file.write('[Common]\n')
        file.write('##########################\n')
        file.write('###################################\n')
        file.write('[Function-1]\n')
        file.write('FilterAndCoherence :\n')
        file.write(f'input : {work_dir}/merged/interferograms/{all_pairs[i]}/fine.int\n')
        file.write(f'filt : {work_dir}/merged/interferograms/{all_pairs[i]}/filt_fine.int\n')
        file.write(f'coh : {work_dir}/merged/interferograms/{all_pairs[i]}/filt_fine.cor\n')
        file.write(f'strength : {strength}\n')
        file.write(f'slc1 : {work_dir}/merged/SLC/{slc_1s[i]}/{slc_1s[i]}.slc.full\n')
        file.write(f'slc2 : {work_dir}/merged/SLC/{slc_2s[i]}/{slc_2s[i]}.slc.full\n')
        file.write(f'complex_coh : {work_dir}/merged/interferograms/{all_pairs[i]}/fine.cor\n')
        file.write(f'range_looks : {range_looks}\n')
        file.write(f'azimuth_looks : {azimuth_looks}\n')

# 在 configs_annual 文件夹中创建文件并写入run_16_内容
for i in range(len(all_pairs)):
    file_name = f'config_igram_unw_{all_pairs[i]}'
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'w') as file:
        file.write('[Common]\n')
        file.write('##########################\n')
        file.write('###################################\n')
        file.write('[Function-1]\n')
        file.write('unwrap :\n')
        file.write(f'ifg : {work_dir}/merged/interferograms/{all_pairs[i]}/filt_fine.int\n')
        file.write(f'unw : {work_dir}/merged/interferograms/{all_pairs[i]}/filt_fine.unw\n')
        file.write(f'coh : {work_dir}/merged/interferograms/{all_pairs[i]}/filt_fine.cor\n')
        file.write('nomcf : False\n')
        file.write(f'reference : {work_dir}/reference\n')
        file.write('defomax :  2\n')
        file.write(f'rlks : {range_looks}\n')
        file.write(f'alks : {azimuth_looks}\n')
        file.write('rmfilter : False\n')
        file.write('method : snaphu\n')   
        file.write('update : True\n') 